### **Connecting with Drive**

In [6]:
#Reading the training data Subject
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Importing All needed libraries**

In [7]:
#Importing all needed libraries
import pandas as pd
import numpy as np #Matric math
import tensorflow as tf #ML
from tensorflow.python.framework import ops
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import sys
import os
from scipy.io import loadmat
from scipy.io import loadmat

# sys.path.append(os.path.abspath("/Users/henda/anaconda3/Lib/site-packages"))
# from rnn_utils import *
# from public_tests import *
ops.reset_default_graph()

tf.compat.v1.reset_default_graph() #Clearning cache
sess=tf.compat.v1.InteractiveSession()




ERROR:tensorflow:An interactive session is already active. This can cause out-of-memory errors or some other unexpected errors (due to the unpredictable timing of garbage collection) in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s). Please use `tf.Session()` if you intend to productionize.


### **Reading the Training dataset**

In [8]:
import pandas as pd
from scipy.io import loadmat

def load_files():
    full_data = pd.DataFrame()
    subjects_to_load = [1,3,5,6,7,8,9,10,11]

    for i in subjects_to_load:
        # Load the .mat file
        training_data = loadmat(f'/content/drive/My Drive/Colab Notebooks/processed withstim/S{i}_E1.mat')
        Data = training_data["Data"]
        Data_df = pd.DataFrame(Data)
        full_data = pd.concat([full_data, Data_df], ignore_index=True)
    return full_data



In [9]:
Data1 = load_files()
Data1

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0.652864,0.530705,0.739539,0.701483,0.528111,0.805447,0.683260,0.356153,0.694081,0.710107,...,0.002965,0.026713,0.009411,0.002163,0.009314,0.003002,0.004143,0.045896,0.174906,0.0
1,0.648498,0.530705,0.738417,0.696472,0.529103,0.806432,0.683260,0.355579,0.691954,0.717659,...,0.003313,0.017410,0.012861,0.002893,0.011303,0.004596,0.004695,0.043423,0.128346,0.0
2,0.648426,0.519349,0.737575,0.701723,0.523797,0.805182,0.681295,0.351392,0.688143,0.712003,...,0.002541,0.012399,0.014314,0.001169,0.006746,0.003317,0.002436,0.050508,0.124272,0.0
3,0.646388,0.530480,0.739259,0.699609,0.531441,0.807466,0.685004,0.355577,0.689305,0.715768,...,0.002981,0.014918,0.009515,0.001943,0.010017,0.004264,0.004030,0.035231,0.138247,0.0
4,0.650530,0.533190,0.739257,0.699825,0.533375,0.808103,0.688155,0.355579,0.704201,0.717659,...,0.003400,0.021507,0.011873,0.001533,0.008065,0.003501,0.003971,0.051130,0.170656,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82022,0.579613,0.634946,0.562432,0.737734,0.651783,0.605426,0.716873,0.559049,0.854303,0.748481,...,0.031994,0.043463,0.093284,0.042815,0.054687,0.052606,0.014951,0.161288,0.089513,17.0
82023,0.608090,0.609223,0.589140,0.723470,0.614642,0.651421,0.697862,0.538956,0.838900,0.728230,...,0.030574,0.033406,0.082356,0.043418,0.043733,0.062681,0.014997,0.180240,0.072349,17.0
82024,0.597995,0.597469,0.615590,0.707872,0.580551,0.695557,0.693106,0.509153,0.813056,0.736915,...,0.042141,0.041796,0.069273,0.046755,0.072634,0.093328,0.025380,0.197048,0.071308,17.0
82025,0.591268,0.574709,0.618580,0.696352,0.556119,0.704743,0.672717,0.497353,0.799376,0.706318,...,0.037183,0.057157,0.129710,0.048599,0.127234,0.063206,0.010009,0.115575,0.070569,17.0


### **Preparing the training dataset**

In [10]:
Data1=np.array(Data1)

# Extract features and targets for training
X_train = Data1[:, 36:58]   # Target for training
Z_train = Data1[:, 58:82]  # Input features for training


del Data1
#####################################################
print(X_train.shape)
print(Z_train.shape)


(82027, 22)
(82027, 24)


### **Reading Test datase**

In [11]:
data_path = '/content/drive/My Drive/Colab Notebooks/processed withstim/S4_E1.mat'


test_data = loadmat(data_path)
Data=test_data["Data"]
Data1=np.array(Data)

print(Data1.shape)

(9108, 83)


In [12]:
X_test = Data1[:, 36:58]  # Target for testing it starts at 36 and ends at 58
Z_test = Data1[:, 58:82]  # Input features for testing
del Data1
print(X_test.shape)
print(Z_test.shape)

(9108, 22)
(9108, 24)


### **Reshaping of training and testing datase**

In [13]:
sequence_length=15
n_features=24
# Prepare input and output for LSTM (training)
n_sequences_train = len(Z_train) - sequence_length + 1
in_train = np.zeros((n_sequences_train, sequence_length, n_features))
out_train = np.zeros((n_sequences_train, X_train.shape[1]))  # Output shape should match target

for j in range(n_sequences_train):
  in_train[j] = Z_train[j:j + sequence_length]
  out_train[j] = X_train[j + sequence_length - 1]  # Target is the last step in the sequence

    # Prepare input and output for LSTM (testing)
n_sequences_test = len(Z_test) - sequence_length + 1
in_test = np.zeros((n_sequences_test, sequence_length, n_features))
out_test = np.zeros((n_sequences_test, X_test.shape[1]))

for j in range(n_sequences_test):
  in_test[j] = Z_test[j:j + sequence_length]
  out_test[j] = X_test[j + sequence_length - 1]  # Target is the last step in the sequence

# Print the shape of the prepared data for sanity check
print(f"LSTM training input shape: {in_train.shape}, LSTM training output shape: {out_train.shape}")
print(f"LSTM testing input shape: {in_test.shape}, LSTM testing output shape: {out_test.shape}")


LSTM training input shape: (82013, 15, 24), LSTM training output shape: (82013, 22)
LSTM testing input shape: (9094, 15, 24), LSTM testing output shape: (9094, 22)


### **Seperate Validation Dataset**

In [14]:
import random

# Define the range and the exclusions
numbers = list(range(1, 41))
exclusions = {28, 2, 3,1}
# Remove the excluded numbers
valid_numbers = [num for num in numbers if num not in exclusions]
# Generate a random number from the valid list
i = random.choice(valid_numbers)

print(f"The validation data is from subject :{i}")

data_path = f'/content/drive/My Drive/Colab Notebooks/processed withstim/S{i}_E1.mat'


val_data = loadmat(data_path)
Data=val_data["Data"]
Data1=np.array(Data)

print(Data1.shape)
X_val = Data1[:, 36:58]   # Target for testing
Z_val = Data1[:, 58:82]  # Input features for testing
del Data1
print(X_val.shape)
print(Z_val.shape)

#prepare validation datase

sequence_length=15
n_features=24
# Prepare input and output for LSTM (training)
n_sequences_val = len(Z_val) - sequence_length + 1
in_val = np.zeros((n_sequences_val, sequence_length, n_features))
out_val = np.zeros((n_sequences_val, X_val.shape[1]))  # Output shape should match target

for j in range(n_sequences_val):
  in_train[j] = Z_val[j:j + sequence_length]
  out_train[j] = X_val[j + sequence_length - 1]  # Target is the last step in the sequence

print(f"val input shape: {in_val.shape}, LSTM val output shape: {out_val.shape}")


The validation data is from subject :29
(8960, 83)
(8960, 22)
(8960, 24)
val input shape: (8946, 15, 24), LSTM val output shape: (8946, 22)


### **Create the Model**

In [15]:
from tensorflow import keras
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, RepeatVector
from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, MaxPooling1D, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.layers import Attention
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras import activations, Input
from tensorflow.keras.models import Model
import tensorflow as tf
import tensorflow.keras.backend as K



# Define our settings
numResponses = 22
numHiddenUnits = 500
timesteps = 15  # Number of timesteps (sequence length)
input_features = 24

class SimpleGNNLayer(tf.keras.layers.Layer):
    def __init__(self, num_features, **kwargs):
        super(SimpleGNNLayer, self).__init__(**kwargs)
        self.num_features = num_features

    def build(self, input_shape):
        # Adjacency matrix for feature-to-feature relationships
        self.adjacency_matrix = self.add_weight(
            shape=(input_shape[-1], self.num_features),  # input_features -> num_features
            initializer="random_normal",
            trainable=True,
            name="adjacency_matrix"
        )

    def call(self, inputs):
        # Compute new features via adjacency matrix multiplication
        # This keeps (batch_size, timesteps, num_features) intact
        return tf.einsum('bti,ij->btj', inputs, self.adjacency_matrix)  # Batch matmul


# Custom Domain-Specific Normalization Layer
class DomainSpecificNormalization(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(DomainSpecificNormalization, self).__init__(**kwargs)

    def build(self, input_shape):
        self.gamma = self.add_weight(
            shape=(1, 1, input_shape[-1]),
            initializer="ones",
            trainable=True,
            name="gamma"
        )
        self.beta = self.add_weight(
            shape=(1, 1, input_shape[-1]),
            initializer="zeros",
            trainable=True,
            name="beta"
        )

    def call(self, inputs):
        mean = tf.reduce_mean(inputs, axis=(1, 2), keepdims=True)
        std = tf.math.reduce_std(inputs, axis=(1, 2), keepdims=True)
        normalized = (inputs - mean) / (std + tf.keras.backend.epsilon())
        return self.gamma * normalized + self.beta


inputs = Input(shape=(timesteps, input_features))

# Domain-Specific Normalization
x = DomainSpecificNormalization()(inputs)

# Convolutional layers
x = Conv1D(32, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

x = Conv1D(64, kernel_size=3, strides=1, padding='same', use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling1D(pool_size=2, strides=1, padding='same')(x)

# GNN Layer (Preserves timesteps)
x = SimpleGNNLayer(num_features=64)(x)

# Bidirectional LSTM layers
x = Bidirectional(LSTM(500, return_sequences=True))(x)
x = Dropout(0.2)(x)

# Multi-Head Attention
attn_output = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = Add()([x, attn_output])  # Residual connection
x = LayerNormalization()(x)

# Dense Layers and Output
x = Dense(400, activation='relu')(x)
x = Dense(200, activation='relu')(x)

# Flatten sequence dimension before final output
x = tf.keras.layers.GlobalAveragePooling1D()(x)

# Final output for DOFs
outputs = Dense(numResponses, activation='linear')(x)

# Define model
model = Model(inputs, outputs)
model.summary()



# ########      LSTM- CNN Hybrid        ##########

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, MaxPooling1D, Dense, Dropout, Bidirectional, LSTM
# from tensorflow.keras import activations

# # Define optimized CNN-LSTM architecture
# model = Sequential()

# # 1st convolutional layer
# model.add(Conv1D(32, kernel_size=3, strides=1, padding='same', use_bias=False, input_shape=(timesteps, input_features)))  # 15 timesteps, 24 features
# model.add(BatchNormalization())
# model.add(Activation(activations.relu))
# model.add(MaxPooling1D(pool_size=2, strides=1, padding='same'))

# # 2nd convolutional layer
# model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation(activations.relu))
# model.add(MaxPooling1D(pool_size=2, strides=1, padding='same'))

# # 3rd convolutional layer
# model.add(Conv1D(128, kernel_size=3, strides=1, padding='same', use_bias=False))
# model.add(BatchNormalization())
# model.add(Activation(activations.relu))
# model.add(MaxPooling1D(pool_size=2, strides=1, padding='same'))


# # Dense layer before LSTM
# model.add(Dense(400, activation='relu'))

# # Bidirectional LSTM layers
# model.add(Bidirectional(LSTM(1000, return_sequences=True)))
# model.add(Dropout(0.2))

# model.add(Bidirectional(LSTM(1000)))
# model.add(Dropout(0.2))
# # Final output layer for predicting 22 degrees of freedom (DOFs)
# model.add(Dense(numResponses, activation='relu'))  # Adjust the number of outputs for your case (22 DOFs)

# # Print model summary
# model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 15, 24)         │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ domain_specific_normaliz… │ (None, 15, 24)         │             48 │ input_layer[0][0]      │
│ (DomainSpecificNormaliza… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 15, 32)         │          2,304 │ domain_specific_norma… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 15, 32)         │            128 │ conv1d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 15, 32)         │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d             │ (None, 15, 32)         │              0 │ activation[0][0]       │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 15, 64)         │          6,144 │ max_pooling1d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 15, 64)         │            256 │ conv1d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 15, 64)         │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling1d_1           │ (None, 15, 64)         │              0 │ activation_1[0][0]     │
│ (MaxPooling1D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ simple_gnn_layer          │ (None, 15, 64)         │          4,096 │ max_pooling1d_1[0][0]  │
│ (SimpleGNNLayer)          │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 15, 1000)       │      2,260,000 │ simple_gnn_layer[0][0] │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 15, 1000)       │              0 │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 15, 1000)       │      1,025,768 │ dropout[0][0],         │
│ (MultiHeadAttention)      │                        │                │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 15, 1000)       │              0 │ dropout[0][0],         │
│                           │                        │                │ multi_head_attention[… │
├──────────────────────

 Total params: 3,785,766 (14.44 MB)

 Trainable params: 3,785,574 (14.44 MB)

 Non-trainable params: 192 (768.00 B)

### **Train the model**

In [16]:
import numpy as np
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

######### Model Training ##########
# Compile the model with Adam optimizer and Mean Squared Error loss
optimizer = Adam(learning_rate=0.001)  # Initial learning rate
model.compile(optimizer=optimizer, loss='mean_squared_error')

# ReduceLROnPlateau callback to dynamically reduce the learning rate if the model stops improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, min_lr=1e-6)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=30)

# Train the model
# history = model.fit(in_train, out_train, validation_split=0.2, batch_size=32, epochs=200, verbose=1, callbacks=[early_stopping, reduce_lr])
history = model.fit(in_train, out_train, validation_data=(in_val,out_val), batch_size=32, epochs=200, verbose=1, callbacks=[early_stopping, reduce_lr])


Epoch 1/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 42s 13ms/step - loss: 0.0261 - val_loss: 0.3762 - learning_rate: 0.0010
Epoch 2/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.0098 - val_loss: 0.3854 - learning_rate: 0.0010
Epoch 3/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.0074 - val_loss: 0.2820 - learning_rate: 0.0010
Epoch 4/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 33s 13ms/step - loss: 0.0060 - val_loss: 0.2702 - learning_rate: 0.0010
Epoch 5/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.0050 - val_loss: 0.2611 - learning_rate: 0.0010
Epoch 6/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 32s 12ms/step - loss: 0.0042 - val_loss: 0.2727 - learning_rate: 0.0010
Epoch 7/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.0038 - val_loss: 0.2835 - learning_rate: 0.0010
Epoch 8/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.0034 - val_loss: 0.2810 - learning_rate: 0.0010
Epoch 9/200
2563/2563 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - loss: 0.0030 - val_lo

### **Test the Model**

In [17]:
# Initialize lists for storing CC and RMSE for 22 DOFs
pearson = []
mse = []

# Model testing
output_predicted = model.predict(in_test)

# Handle sequence length discrepancy if any
n = len(out_test) - len(output_predicted)
if n == 0:
    out_test_final = out_test
else:
    out_test_final = out_test[:-n, :]

# Print shapes for sanity check
print(f"Predicted output shape: {output_predicted.shape}")
print(f"Actual test output shape: {out_test_final.shape}")

# Calculate Pearson correlation and RMSE for each dimension (1 to 22)
for k in range(1, 23):
    # Calculate Pearson correlation coefficient
    cc_value = np.corrcoef(out_test_final[:, k - 1], output_predicted[:, k - 1])[0, 1]

    # Calculate RMSE
    rmse_value = math.sqrt(mean_squared_error(out_test_final[:, k - 1], output_predicted[:, k - 1]))

    # Store values rounded to two decimal places
    pearson.append(round(cc_value, 2))  # Round CC value
    mse.append(round(rmse_value, 2))    # Round RMSE value

    # Print the rounded values for the current DOF
    print(f"Pearson correlation for DOF {k}: {pearson[-1]}")
    print(f"RMSE for DOF {k}: {mse[-1]}")

# Print final lists
print(f"Final Pearson correlation values: {pearson}")
print(f"Final RMSE values: {mse}")


285/285 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Predicted output shape: (9094, 22)
Actual test output shape: (9094, 22)
Pearson correlation for DOF 1: 0.33
RMSE for DOF 1: 0.18
Pearson correlation for DOF 2: 0.04
RMSE for DOF 2: 0.16
Pearson correlation for DOF 3: 0.18
RMSE for DOF 3: 0.34
Pearson correlation for DOF 4: 0.29
RMSE for DOF 4: 0.18
Pearson correlation for DOF 5: 0.39
RMSE for DOF 5: 0.19
Pearson correlation for DOF 6: 0.43
RMSE for DOF 6: 0.27
Pearson correlation for DOF 7: 0.21
RMSE for DOF 7: 0.15
Pearson correlation for DOF 8: 0.28
RMSE for DOF 8: 0.23
Pearson correlation for DOF 9: 0.41
RMSE for DOF 9: 0.23
Pearson correlation for DOF 10: 0.15
RMSE for DOF 10: 0.3
Pearson correlation for DOF 11: 0.31
RMSE for DOF 11: 0.14
Pearson correlation for DOF 12: 0.22
RMSE for DOF 12: 0.18
Pearson correlation for DOF 13: 0.34
RMSE for DOF 13: 0.28
Pearson correlation for DOF 14: -0.1
RMSE for DOF 14: 0.25
Pearson correlation for DOF 15: 0.12
RMSE for DOF 15: 0.13
Pearson correlation f